In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Extract CSVs into DataFrames

In [5]:
police = "C:/Users/Tony Duron/playing_around/police_murders.csv"

police_df = pd.read_csv(police)
police_df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2/1/2015,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2/1/2015,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,3/1/2015,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,4/1/2015,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,4/1/2015,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


In [86]:
county_file = "../Resources/CountyLicenseCount.csv"
county_file_df = pd.read_csv(county_file)

county_file_df

,ID,County Name (Licensee),County ID Code,License Count
0,0,ALBANY,5,77
1,1,ALLEGANY,59,4
2,2,BRONX,0,104
3,3,BROOME,35,14
4,4,CATTARAUGUS,41,9
...,...,...,...,...
57,57,WASHINGTON,15,5
58,58,WAYNE,52,9
59,59,WESTCHESTER,20,73
60,60,WYOMING,56,2


### Transform premise DataFrame

In [87]:
new_premise_file_df = premise_file_df[['License Serial Number', 'Premises Name', 'County ID Code']].copy()
new_premise_file_df.head()

new_premise_file_df.rename(columns = {'License Serial Number':'id','Premises Name':'premise_name', 'County ID Code':'county_id'}, inplace = True)
new_premise_file_df.head()

,id,premise_name,county_id
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
3,1310024,TKO BEVERAGES LLC,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,2


### Transform county DataFrame

In [88]:
new_county_file_df = county_file_df[['ID', 'County Name (Licensee)', 'License Count', 'County ID Code']].copy()
new_county_file_df.head()

new_county_file_df.rename(columns = {'ID':'id', 'County Name (Licensee)':'county_name', 'License Count': 'license_count','County ID Code':'county_id'}, inplace = True)
new_county_file_df.head()

,id,county_name,license_count,county_id
0,0,ALBANY,77,5
1,1,ALLEGANY,4,59
2,2,BRONX,104,0
3,3,BROOME,14,35
4,4,CATTARAUGUS,9,41


### Create database connection

In [89]:
rds_connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [90]:
# Confirm tables
engine.table_names()

['premise_file_df',
 'county_file_df',
 'premise_name',
 'county_name',
 'county_name_1',
 'license-name']

### Load DataFrames into database

In [93]:
new_premise_file_df.to_sql(name='new_premise_file', con=engine, if_exists='append', index=False)

In [94]:
new_county_file_df.to_sql(name='new_county_file', con=engine, if_exists='append', index=False)